# **Chapter 16: Specialized Security Domains**

## Introduction: Beyond Traditional Enterprise Security

In Chapter 15, we architected DevSecOps pipelines for cloud-native applications—automating security for containers, microservices, and AI models that deploy in seconds across elastic infrastructure. However, critical infrastructure and specialized computing environments operate under fundamentally different constraints. A power grid cannot be "patched during the next sprint" if the patch requires downtime. A medical device running on a battery cannot perform TLS handshakes every second without draining power. A smart contract, once deployed to a blockchain, is immutable—there is no "rollback" button when vulnerabilities are discovered.

This chapter explores security domains where the principles of confidentiality, integrity, and availability (CIA) manifest differently than in enterprise IT. In **Operational Technology (OT)** and **Industrial Control Systems (ICS)**, availability reigns supreme—a halted assembly line costs millions per hour, and safety systems must function even when networks are compromised. **Mobile applications** face fragmented ecosystems, sandbox escapes, and the physical loss of devices containing sensitive data. **Internet of Things (IoT)** devices operate with severe resource constraints—kilobytes of memory, intermittent connectivity, and decade-long lifespans that outlast cryptographic standards. **Blockchain and Web3** systems introduce immutable code execution and economic incentives that attract sophisticated adversaries exploiting smart contract logic flaws to steal millions in seconds.

We will navigate the **ISA/IEC 62443** standards for industrial automation security, the **OWASP Mobile Security** framework, lightweight cryptographic protocols for constrained devices, and the **Smart Contract Security Verification Standard (SCSVS)**. By the end, you will understand how to adapt security architectures to environments where traditional patching is impossible, physical safety is paramount, and code immutability demands perfect pre-deployment verification.

---

## 16.1 Industrial Control Systems (ICS) & OT Security

Industrial Control Systems (ICS) encompass Supervisory Control and Data Acquisition (SCADA), Distributed Control Systems (DCS), and Programmable Logic Controllers (PLC). These systems manage physical processes—electricity generation, water treatment, chemical manufacturing, and transportation networks.

### The IT/OT Convergence Problem

Historically, OT networks were **air-gapped**—physically isolated from corporate IT networks and the internet. The Stuxnet attack (2010) demonstrated that air gaps are permeable (via USB drives) and that IT security tools fail in OT environments because:

1. **Availability Priority**: Safety systems must operate continuously; patching requires scheduled maintenance windows months in advance
2. **Legacy Protocols**: Modbus, DNP3, and BACnet were designed for serial networks without authentication or encryption
3. **Long Lifecycles**: ICS equipment operates for 15-20 years, running Windows XP or proprietary OSes no longer supported
4. **Safety Implications**: A cyberattack can cause physical destruction, environmental disasters, or loss of life

### The Purdue Model and Network Segmentation

The **Purdue Model for Industrial Control** (ISA-99) defines six levels of network architecture:

| Level | Name | Components | Security Focus |
|-------|------|------------|----------------|
| **Level 0** | Process | Sensors, actuators, pumps | Physical tampering, signal injection |
| **Level 1** | Basic Control | PLCs, RTUs (Remote Terminal Units) | Firmware integrity, logic validation |
| **Level 2** | Supervisory | HMIs (Human-Machine Interfaces), SCADA servers | Access control, protocol security |
| **Level 3** | Site Operations | Manufacturing Execution Systems (MES), historians | Network segmentation, data diodes |
| **Level 4** | Business Planning | ERP systems, email | Standard enterprise security |
| **Level 5** | Enterprise | Corporate network, internet | Perimeter defense |

**Critical Rule**: Direct communication between Level 4/5 (enterprise) and Level 0/1 (control) should never occur. Data must flow through intermediate layers with protocol translation.

### ISA/IEC 62443 Framework

The **ISA/IEC 62443** series provides standards for securing industrial automation and control systems (IACS). It defines security capabilities across four layers:

**1. General (62443-1-x)**: Concepts, models, and terminology
**2. Policies and Procedures (62443-2-x)**: Security program requirements for asset owners
**3. System (62443-3-x)**: System security requirements and security levels
**4. Component (62443-4-x)**: Product development requirements

**Security Levels (SL) in 62443-3-3**:
- **SL-0**: No specific security requirements
- **SL-1**: Protection against casual/consolidated attackers (script kiddies)
- **SL-2**: Protection against intentional attackers using simple means (hacktivists)
- **SL-3**: Protection against sophisticated attackers with moderate resources (organized crime)
- **SL-4**: Protection against sophisticated attackers with extended resources (nation-states)

### Securing Legacy Protocols

**Modbus TCP** (port 502) is a clear-text protocol with no authentication. An attacker who gains network access can send arbitrary commands:

```python
# Modbus security proxy/inspector
from pymodbus.client import ModbusTcpClient
from pymodbus.exceptions import ModbusException
import logging

class SecureModbusProxy:
    """
    Implements application-layer firewall for Modbus TCP
    Validates function codes and register ranges
    """
    
    ALLOWED_FUNCTION_CODES = [0x01, 0x02, 0x03, 0x04]  # Read-only operations
    CRITICAL_REGISTERS = range(0, 100)  # Safety-critical coils
    
    def __init__(self, plc_ip: str, plc_port: int = 502):
        self.plc = ModbusTcpClient(plc_ip, port=plc_port)
        self.audit_log = []
        
    def validate_request(self, function_code: int, register_address: int, 
                        quantity: int, value: int = None) -> bool:
        """
        Validate Modbus request against security policy
        """
        # Block non-read operations during production hours
        if function_code not in self.ALLOWED_FUNCTION_CODES:
            if self.is_production_hours():
                self.log_security_event("WRITE_ATTEMPT_BLOCKED", 
                                      function_code, register_address)
                return False
        
        # Prevent access to safety-critical registers from remote networks
        if register_address in self.CRITICAL_REGISTERS:
            if not self.is_local_network():
                self.log_security_event("CRITICAL_REGISTER_ACCESS_DENIED",
                                      function_code, register_address)
                return False
        
        # Rate limiting
        if self.is_rate_exceeded():
            return False
            
        return True
    
    def read_coils(self, address: int, count: int = 1):
        """Secure wrapper for Modbus read operation"""
        if not self.validate_request(0x01, address, count):
            raise SecurityException("Modbus request blocked by security policy")
        
        result = self.plc.read_coils(address, count)
        self.log_transaction("READ_COILS", address, count, result)
        return result
    
    def log_security_event(self, event_type: str, function_code: int, address: int):
        """Audit logging for 62443 compliance"""
        event = {
            "timestamp": datetime.utcnow().isoformat(),
            "event": event_type,
            "function_code": hex(function_code),
            "register": address,
            "source_ip": self.get_client_ip(),
            "action": "BLOCKED"
        }
        self.audit_log.append(event)
        logging.warning(f"Security event: {event}")
        
        # Alert SOC if critical
        if event_type == "CRITICAL_REGISTER_ACCESS_DENIED":
            self.send_alert(event)

# Usage in network architecture
proxy = SecureModbusProxy(plc_ip="192.168.1.100")
proxy.ALLOWED_FUNCTION_CODES = [0x01, 0x02, 0x03]  # Read-only mode
coils = proxy.read_coils(0, 10)
```

**DNP3 Secure Authentication**:
Modern implementations use DNP3-SA (Secure Authentication) with challenge-response:

```c
// Conceptual DNP3 Secure Authentication implementation
#include "dnp3/auth.h"

typedef struct {
    uint32_t user_id;
    uint8_t key[32];      // Pre-shared key or derived from PKI
    uint32_t challenge;   // Random nonce
    uint64_t last_timestamp;  // Anti-replay
} DNP3_Security_Context;

bool verify_dnp3_message(DNP3_Security_Context* ctx, 
                        DNP3_Message* msg,
                        uint8_t* hmac) {
    // Verify timestamp (prevent replay attacks)
    if (msg->timestamp <= ctx->last_timestamp) {
        log_security_event("REPLAY_ATTACK_DETECTED");
        return false;
    }
    
    // Verify HMAC-SHA256
    uint8_t computed_hmac[32];
    hmac_sha256(ctx->key, sizeof(ctx->key),
                msg->payload, msg->payload_len,
                computed_hmac);
    
    if (memcmp(computed_hmac, hmac, 32) != 0) {
        log_security_event("AUTHENTICATION_FAILURE");
        return false;
    }
    
    ctx->last_timestamp = msg->timestamp;
    return true;
}
```

### Unidirectional Gateways (Data Diodes)

For high-security environments (SL-3/SL-4), **unidirectional gateways** (data diodes) allow data to flow out of the OT network (for monitoring) but physically prevent data from flowing in:

```python
# Data diode implementation concept
class UnidirectionalGateway:
    """
    Software implementation of data diode principles
    Physical layer uses fiber optics with transmit-only on OT side
    """
    
    def __init__(self):
        self.rx_interface = None  # No receive capability on OT side
        self.tx_interface = "eth1"
        self.buffer = []
        
    def transmit_ot_to_it(self, sensor_data: bytes):
        """
        One-way transmission from OT (Level 1) to IT (Level 4)
        No acknowledgment, no retransmission requests allowed
        """
        # Add sequence number for gap detection (but no retransmission)
        packet = {
            "seq": self.sequence_number,
            "timestamp": time.time(),
            "data": sensor_data,
            "integrity_hash": hashlib.sha256(sensor_data).hexdigest()
        }
        
        # UDP transmission (connectionless, one-way)
        self.udp_send(packet, destination="it-network-monitor")
        self.sequence_number += 1
    
    def receive_from_it(self, data: bytes):
        """
        Explicitly disabled - raises exception if attempted
        """
        raise SecurityException("Inbound traffic blocked by data diode policy")
```

### ICS Incident Response

When ICS systems are compromised, standard IT incident response (isolate and eradicate) may cause physical damage:

```python
class ICSIncidentResponse:
    """
    Specialized incident response for industrial control systems
    """
    
    def handle_malware_detection(self, plc_system: PLCSystem, malware_signature: str):
        """
        Never immediately isolate PLCs controlling physical processes
        """
        # 1. Assess process state
        process_state = plc_system.get_process_state()
        
        if process_state.is_unsafe_to_halt():
            # Enter "controlled operation" mode
            self.enable_manual_override()
            self.notify_operators("Malware detected - manual control required")
            
            # Gradual transition to safe state
            self.execute_controlled_shutdown(plc_system)
        else:
            # Safe to isolate immediately
            self.isolate_plc(plc_system)
        
        # 2. Preserve forensic evidence without stopping process
        self.capture_memory_dump(plc_system, method="non-intrusive")
        self.extract_logs(plc_system, preserve_chain_of_custody=True)
        
        # 3. Analyze malware impact on physical process
        if malware_signature == "TRITON":
            # TRITON targets safety instrumented systems (SIS)
            self.emergency_sis_inspection()
```

---

## 16.2 Mobile Application Security

Mobile applications operate in hostile environments—devices are lost, stolen, jailbroken, or infected with malware. The attack surface includes the app itself, the OS (iOS/Android), inter-process communication (IPC), and backend APIs.

### Platform Security Models

**iOS Security Architecture**:
- **Sandbox**: Each app runs in its own container (`/var/mobile/Containers/Data/Application/UUID/`)
- **Keychain**: Encrypted storage for credentials, using Secure Enclave (hardware-backed on modern devices)
- **Code Signing**: Mandatory code signing with Apple-issued certificates
- **App Transport Security (ATS)**: Enforces TLS 1.2+ by default

**Android Security Architecture**:
- **Sandbox**: Linux UID isolation, SELinux mandatory access control
- **Keystore**: Hardware-backed key storage (StrongBox, TEE)
- **Permissions**: Runtime permission model (Android 6.0+), scoped storage (Android 10+)
- **App Signing**: APK signing v2/v3 with signature scheme validation

### Secure Data Storage

**iOS Keychain Implementation:**

```swift
import Security
import Foundation

class SecureKeychain {
    /**
     Store sensitive data in iOS Keychain
     kSecAttrAccessibleWhenUnlockedThisDeviceOnly ensures data 
     never leaves device (no iCloud backup) and requires device unlock
     */
    static func save(key: String, data: Data) -> OSStatus {
        let query = [
            kSecClass as String: kSecClassGenericPassword as String,
            kSecAttrAccount as String: key,
            kSecValueData as String: data,
            kSecAttrAccessible as String: kSecAttrAccessibleWhenUnlockedThisDeviceOnly,
            kSecAttrService as String: "com.company.secureapp"
        ] as [String: Any]
        
        // Delete existing item first (idempotent)
        SecItemDelete(query as CFDictionary)
        
        return SecItemAdd(query as CFDictionary, nil)
    }
    
    static func load(key: String) -> Data? {
        let query = [
            kSecClass as String: kSecClassGenericPassword as String,
            kSecAttrAccount as String: key,
            kSecReturnData as String: kCFBooleanTrue!,
            kSecMatchLimit as String: kSecMatchLimitOne
        ] as [String: Any]
        
        var dataTypeRef: AnyObject?
        let status = SecItemCopyMatching(query as CFDictionary, &dataTypeRef)
        
        if status == noErr {
            return dataTypeRef as? Data
        }
        return nil
    }
    
    // Store API tokens securely
    static func storeAuthToken(token: String) {
        if let data = token.data(using: .utf8) {
            let status = save(key: "auth_token", data: data)
            if status != noErr {
                fatalError("Failed to secure token in Keychain")
            }
        }
    }
}
```

**Android Keystore with Hardware Security:**

```kotlin
import android.security.keystore.KeyGenParameterSpec
import android.security.keystore.KeyProperties
import java.security.KeyStore
import javax.crypto.Cipher
import javax.crypto.KeyGenerator
import javax.crypto.SecretKey
import javax.crypto.spec.GCMParameterSpec

class SecureStorageManager(private val context: Context) {
    private val ANDROID_KEYSTORE = "AndroidKeyStore"
    private val KEY_ALIAS = "app_master_key"
    
    /**
     * Generate hardware-backed key if available (StrongBox)
     * Fallback to TEE (Trusted Execution Environment)
     */
    private fun generateKey() {
        val keyStore = KeyStore.getInstance(ANDROID_KEYSTORE)
        keyStore.load(null)
        
        if (!keyStore.containsAlias(KEY_ALIAS)) {
            val keyGenerator = KeyGenerator.getInstance(
                KeyProperties.KEY_ALGORITHM_AES, 
                ANDROID_KEYSTORE
            )
            
            val builder = KeyGenParameterSpec.Builder(
                KEY_ALIAS,
                KeyProperties.PURPOSE_ENCRYPT or KeyProperties.PURPOSE_DECRYPT
            )
                .setBlockModes(KeyProperties.BLOCK_MODE_GCM)
                .setEncryptionPaddings(KeyProperties.ENCRYPTION_PADDING_NONE)
                .setKeySize(256)
                .setUserAuthenticationRequired(false) // Set true for biometric binding
            
            // Use StrongBox if available (dedicated hardware security module)
            if (android.os.Build.VERSION.SDK_INT >= android.os.Build.VERSION_CODES.P) {
                builder.setIsStrongBoxBacked(true)
            }
            
            keyGenerator.init(builder.build())
            keyGenerator.generateKey()
        }
    }
    
    fun encrypt(plaintext: String): Pair<ByteArray, ByteArray> {
        val cipher = Cipher.getInstance("AES/GCM/NoPadding")
        cipher.init(Cipher.ENCRYPT_MODE, getKey())
        
        val iv = cipher.iv
        val ciphertext = cipher.doFinal(plaintext.toByteArray(Charsets.UTF_8))
        
        return Pair(iv, ciphertext)
    }
    
    private fun getKey(): SecretKey {
        val keyStore = KeyStore.getInstance(ANDROID_KEYSTORE)
        keyStore.load(null)
        return keyStore.getKey(KEY_ALIAS, null) as SecretKey
    }
}
```

### Certificate Pinning

Prevent Man-in-the-Middle (MitM) attacks by embedding the server's certificate or public key in the app:

**iOS (URLSession Pinning):**

```swift
import Foundation

class PinnedURLSessionDelegate: NSObject, URLSessionDelegate {
    // SHA256 hashes of trusted certificates
    let trustedCertHashes = [
        "sha256/AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA=", // Primary
        "sha256/BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB="  // Backup
    ]
    
    func urlSession(
        _ session: URLSession,
        didReceive challenge: URLAuthenticationChallenge,
        completionHandler: @escaping (URLSession.AuthChallengeDisposition, URLCredential?) -> Void
    ) {
        guard let serverTrust = challenge.protectionSpace.serverTrust,
              let certificateChain = SecTrustCopyCertificateChain(serverTrust) as? [SecCertificate],
              let serverCertificate = certificateChain.first else {
            completionHandler(.cancelAuthenticationChallenge, nil)
            return
        }
        
        // Get certificate data
        guard let serverCertificateData = SecCertificateCopyData(serverCertificate) as Data? else {
            completionHandler(.cancelAuthenticationChallenge, nil)
            return
        }
        
        // Calculate SHA256 hash
        let certHash = sha256(data: serverCertificateData)
        let certHashBase64 = "sha256/" + certHash.base64EncodedString()
        
        // Verify against pinned hashes
        if trustedCertHashes.contains(certHashBase64) {
            let credential = URLCredential(trust: serverTrust)
            completionHandler(.useCredential, credential)
        } else {
            // Certificate pinning failed - possible MitM attack
            logSecurityEvent("CERTIFICATE_PINNING_FAILED", 
                           expected: trustedCertHashes, 
                           received: certHashBase64)
            completionHandler(.cancelAuthenticationChallenge, nil)
        }
    }
    
    private func sha256(data: Data) -> Data {
        var hash = [UInt8](repeating: 0, count: Int(CC_SHA256_DIGEST_LENGTH))
        data.withUnsafeBytes {
            _ = CC_SHA256($0.baseAddress, CC_LONG(data.count), &hash)
        }
        return Data(hash)
    }
}
```

**Android (OkHttp Certificate Pinning):**

```kotlin
import okhttp3.CertificatePinner
import okhttp3.OkHttpClient

fun createSecureHttpClient(): OkHttpClient {
    val certificatePinner = CertificatePinner.Builder()
        .add("api.company.com", "sha256/AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA=")
        .add("api.company.com", "sha256/BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB=")
        .build()
    
    return OkHttpClient.Builder()
        .certificatePinner(certificatePinner)
        .build()
}
```

### Secure Inter-Process Communication (IPC)

Mobile apps often communicate with other apps or system services. Insecure IPC leads to data leakage or privilege escalation.

**Android Content Provider Security:**

```xml
<!-- AndroidManifest.xml -->
<!-- Vulnerable: exported content provider -->
<provider
    android:name=".VulnerableProvider"
    android:authorities="com.company.app.data"
    android:exported="true" /> <!-- DANGEROUS: Any app can access -->

<!-- Secure: internal only -->
<provider
    android:name=".SecureProvider"
    android:authorities="com.company.app.data"
    android:exported="false" /> <!-- Safe: only your app can access -->

<!-- Secure: permission-based -->
<provider
    android:name=".RestrictedProvider"
    android:authorities="com.company.app.sensitive"
    android:exported="true"
    android:readPermission="com.company.app.permission.READ_SENSITIVE"
    android:writePermission="com.company.app.permission.WRITE_SENSITIVE" />
```

**iOS App Groups (Secure):**

```swift
// Share data between app and extension securely
let sharedDefaults = UserDefaults(suiteName: "group.com.company.secureapp")

// Never store sensitive data in shared defaults
// Use Keychain with access group instead
let query: [String: Any] = [
    kSecClass as String: kSecClassGenericPassword,
    kSecAttrAccount as String: "shared_secret",
    kSecValueData as String: secretData,
    kSecAttrAccessGroup as String: "teamID.com.company.secureapp", // App group
    kSecAttrAccessible as String: kSecAttrAccessibleWhenUnlockedThisDeviceOnly
]
```

### OWASP Mobile Top 10 Mitigations

**M1: Improper Platform Usage**
- Use iOS Keychain/Android Keystore, not UserDefaults/SharedPreferences for secrets
- Disable clipboard for sensitive fields

**M2: Insecure Data Storage**
- Enable file encryption (Data Protection on iOS, Jetpack Security on Android)
- Sanitize logs: never log `Log.d("API", "Token: $token")`

**M7: Client Code Quality**
- Validate all inputs from IPC, deep links, and QR codes

```java
// Vulnerable deep link handling
// myapp://open?url=http://evil.com

// Secure implementation
@Override
protected void onCreate(Bundle savedInstanceState) {
    super.onCreate(savedInstanceState);
    
    Uri data = getIntent().getData();
    if (data != null && "myapp".equals(data.getScheme())) {
        String url = data.getQueryParameter("url");
        
        // Validate URL is in allowlist
        if (!isAllowedDomain(url)) {
            Log.w("Security", "Blocked navigation to: " + url);
            return;
        }
        
        // Open in Chrome Custom Tab (isolated from app)
        openInCustomTab(url);
    }
}

private boolean isAllowedDomain(String url) {
    // Strict allowlist, not blacklist
    return url != null && (
        url.startsWith("https://trusted.company.com/") ||
        url.startsWith("https://help.company.com/")
    );
}
```

---

## 16.3 IoT Security: Securing Devices, Gateways, and Data

The Internet of Things encompasses resource-constrained sensors, smart home devices, industrial IoT (IIoT), and medical devices. These devices often have limited CPU (8-32 bit), memory (KB to MB), and power (battery-operated), preventing standard cryptographic implementations.

### Lightweight Cryptography

Standard TLS/SSL is too heavy for constrained devices. **MQTT** (Message Queuing Telemetry Transport) and **CoAP** (Constrained Application Protocol) are common IoT protocols requiring lightweight security.

**MQTT with TLS-PSK (Pre-Shared Keys):**

```python
import ssl
import paho.mqtt.client as mqtt

class SecureIoTDevice:
    def __init__(self, device_id: str, psk: bytes):
        self.device_id = device_id
        self.psk = psk  # 16-64 bytes pre-shared key
        
    def connect_secure(self, broker: str, port: int = 8883):
        client = mqtt.Client(client_id=self.device_id)
        
        # TLS-PSK: No certificates needed, uses pre-shared key
        # Suitable for devices that can't store CA certificates
        ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
        ssl_context.set_ciphers('PSK')
        ssl_context.psk = self.psk
        ssl_context.psk_identity = self.device_id.encode()
        
        client.tls_set_context(ssl_context)
        client.tls_insecure_set(False)  # Verify hostname
        
        # Certificate-based auth for more capable devices
        # client.tls_set(
        #     ca_certs="ca.crt",
        #     certfile="device.crt",
        #     keyfile="device.key",
        #     tls_version=ssl.PROTOCOL_TLSv1_2
        # )
        
        client.connect(broker, port)
        return client
    
    def publish_sensor_data(self, client, data: dict):
        """Publish with QoS 1 (at least once delivery)"""
        payload = json.dumps(data)
        # Encrypt payload at application layer for end-to-end security
        encrypted = self.encrypt_payload(payload)
        client.publish(f"devices/{self.device_id}/data", 
                      encrypted, 
                      qos=1)  # QoS 1 for critical data
```

**CoAP with DTLS (Datagram TLS):**

```c
// CoAP DTLS implementation for constrained devices (Contiki/RIOT OS)
#include "coap.h"
#include "dtls.h"

#define KEY_LENGTH 16

static dtls_context_t *dtls_context;
static const unsigned char psk_key[KEY_LENGTH] = {0x00, 0x01, ...}; // Device-specific key

static int verify_psk(struct dtls_context_t *ctx, 
                     const session_t *session,
                     const unsigned char *id, size_t id_len,
                     const unsigned char *key, size_t key_len) {
    // Verify device identity matches PSK identity
    if (id_len != DEVICE_ID_LEN || memcmp(id, device_id, id_len) != 0) {
        return dtls_alert_fatal_create(DTLS_ALERT_ACCESS_DENIED);
    }
    
    memcpy(key, psk_key, KEY_LENGTH);
    return KEY_LENGTH;
}

void secure_coap_send(coap_packet_t *request) {
    // CoAP over DTLS (CoAPS) on port 5684
    dtls_write(dtls_context, &session, request->data, request->length);
}
```

### Secure Boot and Firmware Updates

IoT devices must verify firmware integrity before execution to prevent persistent malware.

**Secure Boot Process:**

```c
// Simplified secure boot implementation
#include "crypto.h"
#include "flash.h"

#define FIRMWARE_START_ADDR 0x08010000
#define SIGNATURE_ADDR 0x080FF000
#define PUBLIC_KEY_ADDR 0x080FF100

typedef struct {
    uint8_t signature[64];      // ECDSA signature
    uint32_t firmware_size;
    uint32_t firmware_version;
    uint32_t crc32;
} firmware_header_t;

bool verify_and_boot(void) {
    firmware_header_t header;
    flash_read(SIGNATURE_ADDR, &header, sizeof(header));
    
    // 1. Verify firmware version (rollback protection)
    uint32_t current_version = get_current_version();
    if (header.firmware_version < current_version) {
        log_security_event("ROLLBACK_ATTEMPT", header.firmware_version);
        return false; // Prevent downgrade attacks
    }
    
    // 2. Verify CRC32 (integrity check)
    uint32_t computed_crc = crc32_flash(FIRMWARE_START_ADDR, header.firmware_size);
    if (computed_crc != header.crc32) {
        log_security_event("CORRUPT_FIRMWARE");
        return false;
    }
    
    // 3. Verify ECDSA signature (authenticity)
    uint8_t public_key[64];
    flash_read(PUBLIC_KEY_ADDR, public_key, sizeof(public_key));
    
    uint8_t firmware_hash[32];
    sha256_flash(FIRMWARE_START_ADDR, header.firmware_size, firmware_hash);
    
    if (!ecdsa_verify(public_key, firmware_hash, header.signature)) {
        log_security_event("INVALID_SIGNATURE");
        return false;
    }
    
    // 4. Boot verified firmware
    jump_to_firmware(FIRMWARE_START_ADDR);
    return true; // Never reached
}
```

**Over-the-Air (OTA) Update Security:**

```python
class SecureOTAUpdater:
    """
    Secure firmware update mechanism for IoT devices
    """
    
    def __init__(self, device_private_key, server_public_key):
        self.private_key = device_private_key
        self.server_public_key = server_public_key
        
    def download_update(self, update_url: str) -> bytes:
        """
        Download and verify firmware update
        """
        # Download manifest
        manifest = self.download_manifest(update_url)
        
        # Verify manifest signature (prevents malicious update servers)
        if not self.verify_ecdsa_signature(manifest, self.server_public_key):
            raise SecurityException("Invalid manifest signature")
        
        # Check version (anti-rollback)
        current_version = self.get_current_firmware_version()
        if manifest['version'] <= current_version:
            raise SecurityException("Rollback attempt detected")
        
        # Download firmware chunks
        firmware = b''
        for chunk_url in manifest['chunks']:
            chunk = self.download(chunk_url)
            firmware += chunk
        
        # Verify firmware hash
        computed_hash = hashlib.sha256(firmware).digest()
        if computed_hash != manifest['firmware_hash']:
            raise SecurityException("Firmware integrity check failed")
        
        # Decrypt firmware (if encrypted)
        if manifest['encrypted']:
            firmware = self.decrypt_aes_gcm(
                firmware, 
                self.derive_key(manifest['version'])
            )
        
        return firmware
    
    def apply_update(self, firmware: bytes):
        """
        Atomic update with rollback capability
        """
        # Write to staging partition
        self.write_to_flash(firmware, partition="staging")
        
        # Verify staging partition
        if not self.verify_flash_integrity(partition="staging"):
            raise RuntimeError("Staging verification failed")
        
        # Atomic swap (pointer update)
        self.swap_partitions()
        
        # Boot new firmware
        self.reboot()
```

### Device Provisioning and PKI

**Manufacturing-Time Provisioning:**

```python
class DeviceProvisioning:
    """
    Secure device provisioning during manufacturing
    """
    
    def provision_device(self, device_id: str, hardware_id: bytes):
        """
        Inject unique credentials during manufacturing
        """
        # Generate device-specific key pair (ECC P-256 for IoT)
        private_key = ec.generate_private_key(ec.SECP256R1())
        public_key = private_key.public_key()
        
        # Create device certificate signed by manufacturing CA
        subject = x509.Name([
            x509.NameAttribute(NameOID.COMMON_NAME, device_id),
            x509.NameAttribute(NameOID.ORGANIZATION_NAME, "Company IoT"),
            x509.NameAttribute(NameOID.SERIAL_NUMBER, hardware_id.hex())
        ])
        
        cert = x509.CertificateBuilder().subject_name(
            subject
        ).issuer_name(
            self.manufacturing_ca.subject
        ).public_key(
            public_key
        ).serial_number(
            x509.random_serial_number()
        ).not_valid_before(
            datetime.utcnow()
        ).not_valid_after(
            datetime.utcnow() + timedelta(days=10*365)  # 10 year lifetime
        ).add_extension(
            x509.SubjectAlternativeName([x509.DNSName(f"{device_id}.iot.company.com")]),
            critical=False
        ).sign(self.manufacturing_ca_key, hashes.SHA256())
        
        # Securely inject into device (HSM or secure element)
        self.inject_into_secure_element(device_id, private_key, cert)
        
        # Register in device directory
        self.register_device(device_id, cert.public_key(), hardware_id)
        
        return cert
```

---

## 16.4 Blockchain and Web3 Security

Blockchain systems introduce unique security properties: immutability (code cannot be patched), transparency (all transactions are public), and economic incentives (attacks can be profitable). Smart contracts—self-executing code on blockchains—have lost billions to vulnerabilities.

### Smart Contract Vulnerabilities

**Reentrancy Attacks:**
The most famous vulnerability, exploited in The DAO hack (2016), where a function is recursively called before state updates:

```solidity
// VULNERABLE CONTRACT
contract VulnerableBank {
    mapping(address => uint) public balances;
    
    function withdraw() public {
        uint amount = balances[msg.sender];
        require(amount > 0);
        
        // External call BEFORE state update (DANGEROUS)
        (bool success, ) = msg.sender.call{value: amount}("");
        require(success);
        
        // State update happens too late
        balances[msg.sender] = 0;
    }
}

// SECURE CONTRACT (Checks-Effects-Interactions pattern)
contract SecureBank {
    mapping(address => uint) public balances;
    bool internal locked; // Reentrancy guard
    
    modifier nonReentrant() {
        require(!locked, "Reentrant call");
        locked = true;
        _;
        locked = false;
    }
    
    function withdraw() public nonReentrant {
        uint amount = balances[msg.sender];
        require(amount > 0, "Insufficient balance");
        
        // 1. CHECKS (validation)
        // 2. EFFECTS (state changes) - Do this FIRST
        balances[msg.sender] = 0;
        
        // 3. INTERACTIONS (external calls) - Do this LAST
        (bool success, ) = msg.sender.call{value: amount}("");
        require(success, "Transfer failed");
    }
}
```

**Integer Overflow/Underflow:**
Solidity <0.8.0 had no built-in overflow checks:

```solidity
// VULNERABLE (Solidity <0.8.0)
function transfer(address to, uint amount) public {
    require(balances[msg.sender] >= amount);
    balances[msg.sender] -= amount; // Underflow if amount > balance
    balances[to] += amount;         // Overflow if near max uint
}

// SECURE: Use SafeMath or Solidity >=0.8.0
using SafeMath for uint256;

function transfer(address to, uint amount) public {
    balances[msg.sender] = balances[msg.sender].sub(amount); // Reverts on underflow
    balances[to] = balances[to].add(amount);                 // Reverts on overflow
}

// Or with Solidity 0.8.0+ (built-in checks)
function transfer(address to, uint amount) public {
    unchecked { // Only use unchecked when you're certain overflow is impossible
        balances[msg.sender] -= amount;
    }
}
```

**Access Control Vulnerabilities:**

```solidity
// SECURE ACCESS CONTROL PATTERN
import "@openzeppelin/contracts/access/AccessControl.sol";

contract SecureContract is AccessControl {
    bytes32 public constant ADMIN_ROLE = keccak256("ADMIN_ROLE");
    bytes32 public constant MINTER_ROLE = keccak256("MINTER_ROLE");
    
    constructor() {
        _grantRole(DEFAULT_ADMIN_ROLE, msg.sender);
        _grantRole(ADMIN_ROLE, msg.sender);
    }
    
    function mint(address to, uint amount) public onlyRole(MINTER_ROLE) {
        // Only addresses with MINTER_ROLE can execute
        _mint(to, amount);
    }
    
    function criticalFunction() public onlyRole(ADMIN_ROLE) {
        // Critical operations restricted to admins
    }
    
    // Anti-pattern to avoid: tx.origin
    function insecure() public {
        require(tx.origin == owner, "Not owner"); // VULNERABLE to phishing
        // Use msg.sender instead
    }
}
```

### Secure Development Patterns

**Pull Over Push Payments:**
Instead of sending Ether to addresses (which can fail and revert transactions), allow users to withdraw:

```solidity
// SECURE: Pull payment pattern
contract Auction {
    mapping(address => uint) public pendingReturns;
    
    function bid() public payable {
        require(msg.value > highestBid, "Bid too low");
        
        // Record previous bid for withdrawal instead of sending immediately
        if (highestBidder != address(0)) {
            pendingReturns[highestBidder] += highestBid;
        }
        
        highestBidder = msg.sender;
        highestBid = msg.value;
    }
    
    function withdraw() public returns (bool) {
        uint amount = pendingReturns[msg.sender];
        if (amount > 0) {
            pendingReturns[msg.sender] = 0; // Prevent reentrancy
            
            (bool success, ) = msg.sender.call{value: amount}("");
            if (!success) {
                pendingReturns[msg.sender] = amount; // Restore on failure
                return false;
            }
        }
        return true;
    }
}
```

**Oracle Security (Price Manipulation):**
DeFi protocols relying on single price sources are vulnerable to flash loan attacks:

```solidity
// VULNERABLE: Single price oracle
function getCollateralValue() public view returns (uint) {
    return collateralAmount * oracle.getPrice(); // Can be manipulated
}

// SECURE: TWAP (Time-Weighted Average Price) and multi-oracle
function getSecurePrice() public view returns (uint) {
    // Use Chainlink decentralized oracles
    (,int256 price,,,) = chainlinkFeed.latestRoundData();
    require(price > 0, "Invalid price");
    
    // Validate against secondary oracle
    uint256 secondaryPrice = uniswapTWAP.consult(token, 1e18);
    
    // Check deviation
    uint256 deviation = (uint256(price) > secondaryPrice) ? 
        (uint256(price) - secondaryPrice) * 100 / secondaryPrice :
        (secondaryPrice - uint256(price)) * 100 / secondaryPrice;
    
    require(deviation < 5, "Price deviation too high"); // Max 5% difference
    
    return uint256(price);
}
```

### Wallet and Key Management

**Hardware Wallet Integration:**

```javascript
// Web3.js with hardware wallet (Ledger/Trezor)
const Transport = require('@ledgerhq/hw-transport-webusb').default;
const AppEth = require('@ledgerhq/hw-app-eth').default;
const Web3 = require('web3');

async function signWithHardwareWallet(txData) {
    const transport = await Transport.create();
    const eth = new AppEth(transport);
    
    // Get public key from hardware wallet (never leaves device)
    const { address } = await eth.getAddress("44'/60'/0'/0/0", false);
    
    // Create transaction
    const tx = {
        to: txData.to,
        value: Web3.utils.toHex(txData.value),
        data: txData.data,
        gasLimit: Web3.utils.toHex(txData.gas),
        gasPrice: Web3.utils.toHex(txData.gasPrice),
        nonce: Web3.utils.toHex(txData.nonce),
        chainId: 1 // Mainnet
    };
    
    // Serialize transaction for signing
    const unsignedTx = serializeTransaction(tx);
    
    // Sign on hardware device (private key never exposed to computer)
    const signature = await eth.signTransaction(
        "44'/60'/0'/0/0",
        unsignedTx
    );
    
    // Combine signature with transaction
    const signedTx = assembleSignedTx(tx, signature);
    
    // Broadcast
    return web3.eth.sendSignedTransaction(signedTx);
}
```

**Multi-Signature Wallets:**

```solidity
// Gnosis Safe style multi-sig
contract MultiSigWallet {
    address[] public owners;
    uint public required;
    mapping(bytes32 => bool) public executed;
    
    event TransactionSubmitted(bytes32 indexed txHash, address indexed sender);
    event TransactionExecuted(bytes32 indexed txHash);
    
    modifier onlyOwner() {
        require(isOwner(msg.sender), "Not owner");
        _;
    }
    
    function submitTransaction(
        address to, 
        uint value, 
        bytes memory data
    ) public onlyOwner returns (bytes32) {
        bytes32 txHash = keccak256(abi.encode(to, value, data, nonce));
        nonce++;
        
        // Store pending transaction
        pendingTransactions[txHash] = Transaction({
            to: to,
            value: value,
            data: data,
            confirmations: 1,
            executed: false
        });
        
        confirmations[txHash][msg.sender] = true;
        emit TransactionSubmitted(txHash, msg.sender);
        return txHash;
    }
    
    function confirmTransaction(bytes32 txHash) public onlyOwner {
        require(!executed[txHash], "Already executed");
        require(!confirmations[txHash][msg.sender], "Already confirmed");
        
        confirmations[txHash][msg.sender] = true;
        pendingTransactions[txHash].confirmations++;
        
        if (pendingTransactions[txHash].confirmations >= required) {
            executeTransaction(txHash);
        }
    }
    
    function executeTransaction(bytes32 txHash) internal {
        Transaction storage txn = pendingTransactions[txHash];
        require(!txn.executed, "Already executed");
        
        txn.executed = true;
        executed[txHash] = true;
        
        (bool success, ) = txn.to.call{value: txn.value}(txn.data);
        require(success, "Transaction failed");
        
        emit TransactionExecuted(txHash);
    }
}
```

### Consensus and Network Attacks

**51% Attack Mitigation:**
For blockchain operators, detecting consensus attacks:

```python
class ConsensusMonitor:
    def detect_selfish_mining(self, blockchain):
        """
        Detect selfish mining (withholding blocks) or 51% attacks
        """
        recent_blocks = blockchain.get_last_n_blocks(100)
        
        # Check for unusual orphan blocks
        orphan_rate = len(blockchain.get_orphans()) / len(recent_blocks)
        if orphan_rate > 0.1:  # >10% orphan rate is suspicious
            self.alert("High orphan rate detected - possible selfish mining")
        
        # Check mining centralization
        miner_distribution = self.analyze_miner_distribution(recent_blocks)
        if miner_distribution.gini_coefficient > 0.8:
            self.alert("Mining centralization detected")
        
        # Check for chain reorganization depth
        if blockchain.reorg_depth > 6:  # More than 6 blocks reverted
            self.alert("Deep chain reorganization - possible 51% attack")
            self.pause_deposits()  # Protect exchange hot wallet
```

---

## Summary and Transition to Chapter 17

In this chapter, we ventured beyond traditional enterprise security into specialized domains where physical safety, resource constraints, and economic incentives reshape the threat landscape. In **Industrial Control Systems**, you learned that OT security prioritizes availability over confidentiality, that legacy protocols like Modbus lack authentication by design, and that the Purdue Model requires strict network segmentation with unidirectional gateways for high-security environments. The **ISA/IEC 62443** framework provided a structured approach to industrial security, defining Security Levels from casual attackers to nation-state threats.

**Mobile application security** demanded understanding platform-specific architectures—iOS Keychain versus Android Keystore, sandbox escapes via insecure IPC, and certificate pinning to prevent MitM attacks in hostile network environments. You implemented secure storage patterns that resist jailbreak/root access and validated deep links to prevent phishing attacks.

For **IoT devices**, we addressed extreme resource constraints through lightweight cryptography (TLS-PSK, DTLS), secure boot chains that verify firmware signatures in hardware, and over-the-air update mechanisms with anti-rollback protection. The challenge of provisioning millions of devices led to manufacturing-time certificate injection and hardware-backed key storage.

Finally, **Blockchain and Web3 security** introduced immutable code execution where vulnerabilities cannot be patched. You analyzed reentrancy attacks and implemented the Checks-Effects-Interactions pattern, secured price oracles against flash loan manipulation, and architected multi-signature wallets for institutional custody. The transparency of blockchain transactions requires privacy-preserving techniques while maintaining auditability.

However, technical controls alone cannot secure an organization. Security policies must be documented, compliance requirements mapped to controls, and risk quantified in business terms. Audits must verify that implemented controls match documented procedures, and incidents must drive continuous improvement through formal governance structures.

In **Chapter 17: Governance, Risk, and Compliance (GRC)**, we ascend from technical implementation to organizational management. You will learn to implement **ISO/IEC 27001:2022** Information Security Management Systems, navigate regulatory frameworks like **PCI DSS**, **HIPAA**, and **SOC 2**, and establish risk assessment methodologies that translate vulnerability severity into business impact. We will explore how to prepare for external audits, manage third-party risk, and build security metrics that communicate effectively to boards and executives.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='15. devsecops_automation.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../5. compliance_auditing_professional_skills/17. governance_risk_and_compliance_grc.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
